In [2]:
import json
import re
import emoji

# =========================================================
# ⚙️ 参数
# =========================================================
INPUT_FILE = "recipe_corpus_full.json"     # JSONL
OUTPUT_JSON_FILE = "recipes_cleaned.json"  # 输出 JSON 数组

# =========================================================
# 🧩 emoji 统一处理
# =========================================================
emoji_pattern = re.compile(
    "[" 
    "\U0001F600-\U0001F64F"
    "\U0001F300-\U0001F5FF"
    "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FA6F"
    "\U0001FA70-\U0001FAFF"
    "\U00002700-\U000027BF"
    "\U0001F1E0-\U0001F1FF"
    "]+",
    flags=re.UNICODE
)

def replace_emojis(s: str) -> str:
    if not isinstance(s, str):
        return s
    prev = {"last": None}
    def repl(match):
        em = match.group(0)[0]
        name = emoji.demojize(em, language="zh")
        name = re.sub(r'^:+|:+$', '', name)
        name = re.sub(r'[_]+', '', name).strip()
        if not name or name == em:
            name = emoji.demojize(em)
            name = re.sub(r'^:+|:+$', '', name)
            name = re.sub(r'[_]+', '', name).strip()
        if not name:
            return ''
        if name == prev["last"]:
            return ''
        prev["last"] = name
        return name
    out = re.sub(emoji_pattern, repl, s)
    out = re.sub(r'\s+', ' ', out).strip()
    return out

# =========================================================
# 🧹 文本清洗
# =========================================================
def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return s

    s = s.replace("\\n", "\n")
    s = re.sub(r'[\b\r\t]', '', s)
    s = re.sub(r'\\"+', '"', s)
    s = re.sub(r'""+', '', s)
    s = re.sub(r';{2,}', ';', s)

    s = replace_emojis(s)

    # 删除独立“图片”或“图片图片”行
    s = re.sub(r'^\s*图片\s*图片?\s*$', '', s, flags=re.MULTILINE)

    # 删除乱码，只保留常规文字、标点、数字
    s = re.sub(r'[^\u4e00-\u9fa5A-Za-z0-9，。、“”‘’！；：：《》〈〉·,.!?()（）\s-]', '', s)

    # 合并多个空格为一个
    s = re.sub(r'\s+', ' ', s)
    # 去掉标点前后的空格
    s = re.sub(r'\s*([，。！？：；,.!?()（）])\s*', r'\1', s)
    s = s.strip()
    return s

# =========================================================
# 🧹 清理列表元素
# =========================================================
def clean_list_item(x: str) -> str:
    if not isinstance(x, str):
        return x
    x = clean_text(x)
    if "成品" in x.replace(" ", "") or "看图文中的做法" in x.replace(" ", ""):
        return ""
    return x

# =========================================================
# 🔥 流式处理
# =========================================================
output_list = []
count = 0

with open(INPUT_FILE, "r", encoding="utf-8") as fin:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except Exception as e:
            print(f"❌ JSON解析失败：{e} | 行片段: {line[:80]}")
            continue

        obj.pop("author", None)

        for k, v in obj.items():
            if isinstance(v, str):
                obj[k] = clean_text(v)
            elif isinstance(v, list):
                obj[k] = [clean_list_item(x) for x in v if clean_list_item(x)]

        # dish 兜底
        dish = (obj.get("dish") or "").strip()
        if not dish or dish.lower() == "unknown":
            obj["dish"] = obj.get("name", "").strip()

        # 构建 text 字段（去掉换行符，全部连成一段）
        parts = []

        desc = obj.get("description", "").strip()
        if desc:
            parts.append(f"描述：{desc}")

        ingredients = obj.get("recipeIngredient", [])
        if ingredients:
            parts.append("配料：" + " ".join(ingredients))

        steps_list = obj.get("recipeInstructions", [])
        if steps_list:
            steps_lines = [f"{i+1}.{x}" for i, x in enumerate(steps_list)]
            parts.append("步骤：" + " ".join(steps_lines))

        keywords = obj.get("keywords", [])
        if keywords:
            parts.append("关键词：" + "，".join(keywords))

        text_block = " ".join(parts)
        text_block = re.sub(r'\s+', ' ', text_block).strip()

        # 🔹 删除空括号（中英文）
        text_block = re.sub(r'（\s*）|\(\s*\)', '', text_block)

        output_list.append({
            "id": count + 1,
            "name": obj.get("name", "").strip(),
            "text": text_block
        })
        count += 1

# 写出 JSON 文件
with open(OUTPUT_JSON_FILE, "w", encoding="utf-8") as fout:
    json.dump(output_list, fout, ensure_ascii=False, indent=2)

print(f"🎉 完成！共处理 {count} 条记录。输出文件：{OUTPUT_JSON_FILE}")


🎉 完成！共处理 1550151 条记录。输出文件：recipes_cleaned.json
